In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

从2024年9月15日开始，您只能通过切换到Vertex AI Gemini提示和调整来自定义分类、实体提取和情感分析模型。针对Vertex AI AutoML用于文本分类、实体提取和情感分析目标的训练或更新模型将不再提供。您可以继续使用现有的Vertex AI AutoML文本目标直到2025年6月15日。有关Gemini如何通过改进的提示功能提供增强用户体验的更多信息，请参见【调整介绍】。

# 为批量预测训练文本实体提取模型的AutoML

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl_text_entity_extraction_batch_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl_text_entity_extraction_batch_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/automl_text_entity_extraction_batch_prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概览

本教程演示了如何使用Vertex AI SDK创建文本实体提取模型，以及如何使用Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) 模型进行批量预测。

了解有关[文本数据的实体提取](https://cloud.google.com/vertex-ai/docs/training-overview#entity_extraction_for_text) 的更多信息。

### 目标

在本教程中，您将从 Python 脚本中创建一个自动ML文本实体提取模型，然后使用 Vertex AI SDK 进行批量预测。您也可以使用 `gcloud` 命令行工具或在 Cloud Console 上在线创建和部署模型。

执行的步骤包括：

- 创建一个 Vertex `Dataset` 资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行即时预测，并实时返回结果。

* 批量预测服务：对整个实例集进行排队（批量）预测，并在完成后将结果存储在 Cloud Storage 存储桶中。

### 数据集

本教程使用的数据集是来自[国家生物技术信息中心](https://www.ncbi.nlm.nih.gov/CBBresearch/Dogan/DISEASE/)的[NCBI疾病研究摘要数据集](https://www.ncbi.nlm.nih.gov/)。您在本教程中将使用的数据集版本存储在一个公共云存储桶中。

### 成本

本教程使用谷歌云的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[价格计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

### 安装

安装以下软件包以执行此笔记本。

In [ ]:
# install packages
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                    jsonlines -q
! pip3 install --upgrade tensorflow -q

仅限 Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### 开始之前

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法:
- 运行 `gcloud config list`
- 运行 `gcloud projects list`
- 查看支持页面: [查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# set the project id
! gcloud config set project $PROJECT_ID

#### 区域

您可以更改 Vertex AI 使用的 `REGION` 变量。
了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行验证。请按照下面的相关指示操作。

**1. Vertex AI工作台** 
- 无需进行任何操作，因为您已经通过验证。

**2. 本地JupyterLab实例，请取消注释并运行。**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

Service Account 或其他
- 在此处查看所有身份验证选项：[Google Cloud Platform Jupyter Notebook 身份验证指南](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_authentication_guide.ipynb)

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时，才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

### 设置变量

接下来，设置一些在整个教程中使用的变量。
### 导入库和定义常数

In [ ]:
import google.cloud.aiplatform as aiplatform

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

教程

现在你已经准备好开始创建自己的AutoML文本实体提取模型。

云存储训练数据的位置。

现在将变量 `IMPORT_FILE` 设置为云存储中JSONL索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/language/ucaip_ten_dataset.jsonl"

快速查看您的数据

本教程使用存储在公共云存储桶中的NCBI生物医学数据集版本，使用JSONL索引文件。

首先快速浏览数据。通过计算JSONL索引文件中对象的数量（`wc -l`）来统计示例的数量，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

创建数据集

接下来，使用`TextDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟的时间。

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name="NCBI Biomedical",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.extraction,
)

print(dataset.resource_name)

### 创建并运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，和2）运行该管道。

#### 创建训练管道

使用`AutoMLTextTrainingJob`类创建一个AutoML训练管道，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：要为之训练模型的任务类型。
  - `classification`：文本分类模型。
  - `sentiment`：文本情感分析模型。
  - `extraction`：文本实体提取模型。
- `multi_label`：如果是分类任务，则是单标签（False）还是多标签（True）。
- `sentiment_max`：如果是情感分析任务，则是最大情感值。

实例化的对象是训练管道的DAG（有向无环图）。

In [ ]:
dag = aiplatform.AutoMLTextTrainingJob(
    display_name="biomedical", prediction_type="extraction"
)

print(dag)

#### 运行训练流程

接下来，您可以通过调用`run`方法来运行DAG以启动训练作业，传入以下参数：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：经过训练的模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。

当完成`run`方法后，将返回`Model`资源。

训练流程的执行将需要最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="biomedical",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## 检查模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations()`方法来查看它的评估分数。这个方法将为每个评估分片返回一个迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

发送批量预测请求

向部署模型发送批量预测。

### 创建测试项目

您可以将合成数据用作测试数据项。不必担心我们使用合成数据 - 我们只是想说明如何进行预测。

In [ ]:
test_item_1 = 'Molecular basis of hexosaminidase A deficiency and pseudodeficiency in the Berks County Pennsylvania Dutch.\tFollowing the birth of two infants with Tay-Sachs disease ( TSD ) , a non-Jewish , Pennsylvania Dutch kindred was screened for TSD carriers using the biochemical assay . A high frequency of individuals who appeared to be TSD heterozygotes was detected ( Kelly et al . , 1975 ) . Clinical and biochemical evidence suggested that the increased carrier frequency was due to at least two altered alleles for the hexosaminidase A alpha-subunit . We now report two mutant alleles in this Pennsylvania Dutch kindred , and one polymorphism . One allele , reported originally in a French TSD patient ( Akli et al . , 1991 ) , is a GT-- > AT transition at the donor splice-site of intron 9 . The second , a C-- > T transition at nucleotide 739 ( Arg247Trp ) , has been shown by Triggs-Raine et al . ( 1992 ) to be a clinically benign " pseudodeficient " allele associated with reduced enzyme activity against artificial substrate . Finally , a polymorphism [ G-- > A ( 759 ) ] , which leaves valine at codon 253 unchanged , is described'
test_item_2 = "Analysis of alkaptonuria (AKU) mutations and polymorphisms reveals that the CCC sequence motif is a mutational hot spot in the homogentisate 1,2 dioxygenase gene (HGO).	We recently showed that alkaptonuria ( AKU ) is caused by loss-of-function mutations in the homogentisate 1 , 2 dioxygenase gene ( HGO ) . Herein we describe haplotype and mutational analyses of HGO in seven new AKU pedigrees . These analyses identified two novel single-nucleotide polymorphisms ( INV4 + 31A-- > G and INV11 + 18A-- > G ) and six novel AKU mutations ( INV1-1G-- > A , W60G , Y62C , A122D , P230T , and D291E ) , which further illustrates the remarkable allelic heterogeneity found in AKU . Reexamination of all 29 mutations and polymorphisms thus far described in HGO shows that these nucleotide changes are not randomly distributed ; the CCC sequence motif and its inverted complement , GGG , are preferentially mutated . These analyses also demonstrated that the nucleotide substitutions in HGO do not involve CpG dinucleotides , which illustrates important differences between HGO and other genes for the occurrence of mutation at specific short-sequence motifs . Because the CCC sequence motifs comprise a significant proportion ( 34 . 5 % ) of all mutated bases that have been observed in HGO , we conclude that the CCC triplet is a mutational hot spot in HGO ."

### 创建批量输入文件

现在创建一个批量输入文件，您将存储在您的本地云存储桶中。批量输入文件只能是JSONL格式。对于JSONL文件，您为每个数据项（实例）的每一行制作一个字典条目。字典包含键/值对：

- `content`：具有文本项的文件的云存储路径。
- `mime_type`：内容类型。在我们的示例中，它是一个`text`文件。

例如：

                        {'content': '[your-bucket]/file1.txt', 'mime_type': 'text'}

In [ ]:
import json

import tensorflow as tf

gcs_test_item_1 = BUCKET_URI + "/test1.txt"
with tf.io.gfile.GFile(gcs_test_item_1, "w") as f:
    f.write(test_item_1 + "\n")
gcs_test_item_2 = BUCKET_URI + "/test2.txt"
with tf.io.gfile.GFile(gcs_test_item_2, "w") as f:
    f.write(test_item_2 + "\n")

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": gcs_test_item_1, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")
    data = {"content": gcs_test_item_2, "mime_type": "text/plain"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

发出批量预测请求

现在您的模型资源已经训练好了，您可以通过调用batch_predict()方法来进行批量预测，参数如下：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批处理请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `sync`：如果设置为True，则调用将在等待异步批处理作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="biomedical",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

### 等待批处理预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的云存储输出桶中。您可以调用iter_outputs()方法获取包含结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
- `ids`：每个预测请求的内部分配唯一标识符。
- `displayNames`：每个类别标签的类名。
- `confidences`：每个类别标签的预测置信度，介于0和1之间。
- `textSegmentStartOffsets`：实体在文本中的起始字符偏移量。
- `textSegmentEndOffsets`：实体在文本中的结束字符偏移量。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

清理

要清理在这个项目中使用的所有Google Cloud资源，您可以删除您在教程中使用的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI

# Delete batch
batch_predict_job.delete()

# Delete model
model.delete()

# Delete text dataset
dataset.delete()

# Delete training job
dag.delete()